# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("Results/city_weather.csv")
weather_df.dropna(inplace = True) 

weather_df

,Unnamed: 0,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country
0,0,41.37,-73.41,63.37,100,90,4.70,5282297,Bethel,US
1,1,35.54,106.69,59.36,32,0,6.80,1798760,Pingliang,CN
2,2,0.60,-80.03,73.96,81,100,6.22,3653967,Muisne,EC
3,3,-17.71,-39.25,73.80,87,93,3.40,3466980,Caravelas,BR
4,4,52.75,87.75,53.91,56,100,3.13,1491197,Spassk,RU
...,...,...,...,...,...,...,...,...,...,...
570,570,-9.66,20.39,70.99,83,87,4.99,145531,Saurimo,AO
571,571,-38.38,142.48,60.01,77,100,13.49,2144528,Warrnambool,AU
572,572,-10.45,161.92,81.57,76,100,5.44,2178753,Kirakira,SB
573,573,29.11,21.29,66.47,62,0,8.01,88533,Awjilah,LY


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Store latitude and longitude in locations
locations = weather_df[["Lat", "Long"]]

# Fill any NaN values and convert to float
humidity = weather_df["Humidity (%)"].astype(float)

#find min and max humidity values
min_humid = weather_df["Humidity (%)"].min()
max_humid = weather_df["Humidity (%)"].max()
print(min_humid)
print(max_humid)

4
100


In [111]:
# Plot Heatmap
fig1 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig1.add_layer(heat_layer)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
weather_df.head(0)

,Unnamed: 0,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country


In [31]:
# * A max temperature lower than 80 degrees but higher than 70.
#   * Wind speed less than 10 mph.
#   * Zero cloudiness.
#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_df = weather_df.loc[((weather_df["Temp (F)"] > 70) & (weather_df["Temp (F)"] < 80)) 
                          & (weather_df["Wind Speed (mph)"] < 10) 
                          & (weather_df["Cloudiness (%)"] == 0), :]
ideal_df = ideal_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [58]:
# export to csv, and import csv back to df, for consistency and recordkeeping
hotel_cities = ideal_df.to_csv("Results_Vpy/hotel_cities.csv")
hotel_df = pd.read_csv("Results_VPy/hotel_cities.csv")

In [59]:
# # Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ''
hotel_df

,Unnamed: 0,Unnamed: 0.1,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country,Hotel Name
0,13,13,-33.59,26.89,71.80,58,0,8.75,964432,Port Alfred,ZA,
1,50,50,27.85,-105.48,70.66,35,0,7.52,3995142,Naica,MX,
2,53,53,22.89,-109.91,79.00,75,0,1.01,3985710,Cabo San Lucas,MX,
3,102,102,29.33,113.09,74.73,44,0,8.90,1927639,Yueyang,CN,
4,125,125,24.69,46.72,78.80,16,0,9.17,108410,Riyadh,SA,
5,129,129,15.80,-84.28,78.98,88,0,8.05,3614835,Barra Patuca,HN,
6,146,146,25.47,-108.10,78.49,82,0,1.99,4005297,Guamúchil,MX,
7,238,238,-34.00,24.73,73.99,45,0,3.00,986717,Kruisfontein,ZA,
8,316,316,20.50,-10.07,79.18,22,0,3.13,2381972,Adrar,MR,
9,323,323,-34.05,23.37,72.64,35,0,2.46,964712,Plettenberg Bay,ZA,


In [60]:
# find the closest hotel to each coordinate set. note, default units for distance are metric (meters) 

params = {"type": "lodging","keyword": "Hotel", "radius": 5000, "key": g_key} 

# use iterrows to iterate through coordinate pairs within pandas dataframe
for x,y in hotel_df.iterrows():
    lat = y["Lat"]
    long = y["Long"]
    
    #add location param to params for each iteration of coordinates
    params["location"] = f"{lat}, {long}"
    
    #establish url for each iteration of coordinates
    #basic url construction: https://maps.googleapis.com/maps/api/place/findplacefromtext/output?parameters
    #output is json or xml, we choose json
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()
    results = response['results']
        
    try:
        hotel_df.loc[x, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
     
        pass
    

hotel_df

,Unnamed: 0,Unnamed: 0.1,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country,Hotel Name
0,13,13,-33.59,26.89,71.80,58,0,8.75,964432,Port Alfred,ZA,Royal St Andrews Hotel
1,50,50,27.85,-105.48,70.66,35,0,7.52,3995142,Naica,MX,
2,53,53,22.89,-109.91,79.00,75,0,1.01,3985710,Cabo San Lucas,MX,Waldorf Astoria Los Cabos Pedregal
3,102,102,29.33,113.09,74.73,44,0,8.90,1927639,Yueyang,CN,Pullman Hotel
4,125,125,24.69,46.72,78.80,16,0,9.17,108410,Riyadh,SA,Four Seasons Hotel Riyadh At Kingdom Center
5,129,129,15.80,-84.28,78.98,88,0,8.05,3614835,Barra Patuca,HN,
6,146,146,25.47,-108.10,78.49,82,0,1.99,4005297,Guamúchil,MX,Hotel Davimar
7,238,238,-34.00,24.73,73.99,45,0,3.00,986717,Kruisfontein,ZA,
8,316,316,20.50,-10.07,79.18,22,0,3.13,2381972,Adrar,MR,
9,323,323,-34.05,23.37,72.64,35,0,2.46,964712,Plettenberg Bay,ZA,Sky Villa Boutique Hotel


In [104]:
hotel2_df = hotel_df.copy()

In [105]:
# drop locations with no hotel within 5000 meters 
hotel2_df["Hotel Name"].replace('', np.nan, inplace=True)
hotel2_df = hotel2_df[hotel2_df['Hotel Name'].notna()]

#hotel2_df.dropna(subset=["Hotel Name"], inplace=True)
hotel2_df

,Unnamed: 0,Unnamed: 0.1,Lat,Long,Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),City ID,City Name,Country,Hotel Name
0,13,13,-33.59,26.89,71.80,58,0,8.75,964432,Port Alfred,ZA,Royal St Andrews Hotel
2,53,53,22.89,-109.91,79.00,75,0,1.01,3985710,Cabo San Lucas,MX,Waldorf Astoria Los Cabos Pedregal
3,102,102,29.33,113.09,74.73,44,0,8.90,1927639,Yueyang,CN,Pullman Hotel
4,125,125,24.69,46.72,78.80,16,0,9.17,108410,Riyadh,SA,Four Seasons Hotel Riyadh At Kingdom Center
6,146,146,25.47,-108.10,78.49,82,0,1.99,4005297,Guamúchil,MX,Hotel Davimar
9,323,323,-34.05,23.37,72.64,35,0,2.46,964712,Plettenberg Bay,ZA,Sky Villa Boutique Hotel
10,366,366,29.45,55.68,71.60,14,0,7.02,114259,Sīrjān,IR,Tourism Hotel
12,391,391,13.38,-16.75,77.00,100,0,8.05,2413753,Brufut,GM,Sheraton Gambia Hotel Resort & Spa
13,393,393,16.86,-99.89,78.01,83,0,4.27,3533462,Acapulco de Juárez,MX,Dreams Acapulco Resort & Spa
14,406,406,33.34,44.40,78.80,44,0,9.17,98182,Baghdad,IQ,Andalus Hotel suites


In [113]:
#export to csv as a backup
hotels = hotel2_df.to_csv("Results_Vpy/hotels.csv")

In [114]:
# NOTE: Do not change any of the code in this cell
coordinates = hotel2_df[["Lat", "Long"]]

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in hotel2_df.iterrows()]


In [115]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(coordinates)
symbols = gmaps.symbol_layer(coordinates, info_box_content = hotel_info)

fig1.add_layer(markers)
fig1.add_layer(symbols)

# Display figure
fig1

Figure(layout=FigureLayout(height='420px'))